In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
import scipy
from pyvis.network import Network


In [48]:
# narrowed down: only edges with >100 count
edgelist_df_small = pd.read_csv("edgelist_df_small_fandoms.csv")
fandoms_small = pd.read_csv("fandoms_small.csv")

In [49]:
len(edgelist_df_small)

3444

In [50]:
# create dict mapping of ids to fandom names to secure node attributes
id_fandom_mapping = dict(fandoms_small[['id', 'name']].values)

# create dict mapping of id to fandom cached_count for nodes
id_cached_count_mapping = dict(fandoms_small[['id', 'cached_count']].values)

# need to adjust cached_count because too big distance between numbers
adjusted_cached_count_mapping = {k: np.log1p(v) for k, v in id_cached_count_mapping.items()}

In [51]:
G_ego = nx.from_pandas_edgelist(edgelist_df_small, source='integer_1', target='integer_2', edge_attr='count')


In [52]:
# set node attribudes for fandom names
nx.set_node_attributes(G_ego, name='fandom', values=id_fandom_mapping)

# i want to color by cached_count of each fandom
nx.set_node_attributes(G_ego, name='cached_count', values=id_cached_count_mapping)

# need to adjust cached_count because too big distance between numbers
nx.set_node_attributes(G_ego, name='adjusted_cached_count', values=adjusted_cached_count_mapping)

# node attributes for degree 
degrees_all = dict(nx.degree(G_ego))
nx.set_node_attributes(G_ego, name='degree', values=degrees_all)

# Slightly adjust degree so that the nodes with very small degrees are still visible
number_to_adjust_by = 10
adjusted_node_size = dict([(node, degree/number_to_adjust_by) for node, degree in nx.degree(G_ego)])
nx.set_node_attributes(G_ego, name='adjusted_node_size', values=adjusted_node_size)

In [57]:
# edge weights / distance
nx.set_edge_attributes(G_ego, nx.get_edge_attributes(G_ego, 'count'), 'weight')
def custom_distance(u, v, edge_data):
    return edge_data['weight'] * 0.01

In [58]:
# GRAPH: CHOOSE EGO FANDOM
ego_fandom = 65
ego_graph = nx.ego_graph(G_ego, ego_fandom, radius=2, distance=custom_distance)
degrees_ego = dict(nx.degree(G_ego))

In [78]:
# Initialize Pyvis Network
net = Network(notebook=True, height="750px", width="100%", bgcolor="white", font_color="black", cdn_resources='in_line')

# Set the layout
pos_ego = nx.circular_layout(ego_graph, scale=1, center=(0, 0))
net.repulsion(node_distance=420, central_gravity=0.33, spring_length=110, spring_strength=0.10, damping=0.95)

ego_graph_limited = ego_graph.subgraph(list(ego_graph.nodes())[:50])


In [79]:
# net.from_nx(ego_graph)
# print(net)

In [80]:
# Check the number of nodes and edges
print(f"Number of nodes in ego graph: {ego_graph.number_of_nodes()}")
print(f"Number of edges in ego graph: {ego_graph.number_of_edges()}")

Number of nodes in ego graph: 6
Number of edges in ego graph: 8


In [81]:
# Example: Precompute values outside the loop
sizes = {node: data['adjusted_node_size'] for node, data in ego_graph_limited.nodes(data=True)}
fandom_name = {node: data['fandom'] for node, data in ego_graph_limited.nodes(data=True)}
colors = {node: 'red' if node == ego_fandom else 'black' for node in ego_graph_limited.nodes()}

In [93]:
# Add nodes with attributes
for node, data in ego_graph_limited.nodes(data=True):
    size = sizes[node]  # Adjust node size
    # color = colors[node]  # Ego node color vs others
    title = fandom_name[node]
    net.add_node(node, label=data['fandom'], title=title, size=size)

# Add edges with weights
for source, target, data in ego_graph_limited.edges(data=True):
    net.add_edge(source, target, value=data['count'])

# net.prep_notebook()
# net.show("net.html")
# display(HTML('net.html'))

In [ ]:
#Choose a title!
title = 'Ego Network of ' + edgelist_df_small[edgelist_df_small['integer_1'] == ego_fandom]['name_1'].iloc[0]

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
        ("Fandom", "@fandom"),
        ("Degree", "@degree"),
        ("Count", "@cached_count")
]

size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'adjusted_cached_count'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Reds8

#Create a plot — set dimensions, toolbar, and title
plot_ego = figure(tooltips = HOVER_TOOLTIPS,
            #   tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-3.5, 3.5), y_range=Range1d(-3.5, 3.5), title=title)

# Define the layout
pos_ego = nx.spring_layout(ego_graph, scale=3, center=(0, 0))
pos_ego[ego_fandom] = (0, 0)

# Convert the NetworkX graph to a Bokeh graph
graph_ego = from_networkx(ego_graph, pos_ego)

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color_ego = min(graph_ego.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color_ego = max(graph_ego.node_renderer.data_source.data[color_by_this_attribute])
color_map = linear_cmap(color_by_this_attribute, color_palette, maximum_value_color_ego, minimum_value_color_ego)
# node_colors = ['blue' if node == ego_fandom else color_map[degrees_ego[node]] for node in ego_graph.nodes()]

graph_ego.node_renderer.glyph = Circle(radius=0.3, fill_color=color_map, fill_alpha=0.7)
graph_ego.edge_renderer.glyph = MultiLine(line_alpha=1, line_width=1, line_color="black")

plot_ego.renderers.append(graph_ego)
show(plot_ego)
